In [1]:
import datetime
import _speed_utils as speed_utils
import _threshold_utils as threshold_utils
import altair as alt
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, segment_calcs
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
    analysis_date,
    CONFIG_PATH
)
from shared_utils import calitp_color_palette as cp

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/jovyan/data-analyses/rt_segment_speeds/_threshold_utils.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/lat

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# alt.data_transformers.disable_max_rows()

### Merging

In [4]:
def merge_all_speeds(analysis_date:str) -> pd.DataFrame:
    """
    Merge avg_speeds_stop_segments and
    speed_stops parquets.
    
    Args:
        date: analysis date
    """
    # Open up avg speeds
    avg_speeds = pd.read_parquet(f"{speed_utils.GCS_PATH}avg_speeds_stop_segments_{analysis_date}.parquet")
    avg_speeds = avg_speeds.drop(columns=["geometry", "geometry_arrowized", "district", "district_name"])
    # Filter  for all day flags
    avg_speeds = avg_speeds[avg_speeds.time_of_day == 'all_day'].reset_index(drop = True)
    
    # Open up speeds
    speeds = pd.read_parquet(f"{speed_utils.GCS_PATH}speeds_stop_segments_{analysis_date}")
    
    merge_cols = ['gtfs_dataset_key','shape_array_key', 'stop_sequence']
    m1 = pd.merge(avg_speeds, speeds, on = merge_cols, how = 'inner')
    
    m1 = m1.drop_duplicates().reset_index(drop = True)
    
    return m1

In [5]:
m1 = merge_all_speeds(analysis_date)

In [6]:
# m1.shape

In [7]:

# Picked 4 random routes
sample_0_keys = [
    "0fb4f3627996269dc7075276d3b69e36",
    "07c9a47264a43d8d0d16ef7109e8fd68",
    "106d979b9a9e6338827a8e1c145e69fd",
    "000624bd8453dbe4f2eb2765b04bcb98",
]

### Categorize

In [8]:
def categorize_by_percentile_pandas(
    df: pd.DataFrame, column_percentile: str, column_str: str
) -> pd.DataFrame:

    # Find percentiles
    #Get percentiles in objects for total vehicle.
    p5 = df[column_percentile].quantile(0.05).astype(float)
    p95 = df[column_percentile].quantile(0.95).astype(float)
    
    def rate(row):
        if ((row[column_percentile] >= 0) and (row[column_percentile] <= p5)):
            return f"{column_str} is low"
        elif (row[column_percentile] >= p95):
               return f"{column_str} is high"
        else:
            return f"{column_str} is avg"
    
    # Apply flags
    df[f"{column_str}cat"] = df.apply(lambda x: rate(x), axis=1)
    
    # Clean
    df[f"{column_str}cat"] = df[f"{column_str}cat"].str.replace("_", "")

    print(f"Done with {column_str}")
    
    return df  

In [9]:
# df1 = categorize_by_percentile_pandas(subset, "meters_elapsed", "meters_")

In [10]:
# df1.head()

In [11]:
# df2 = categorize_by_percentile_pandas(df1, "sec_elapsed", "sec_")

In [12]:
# df2.head()

In [13]:
def categorize_meters_speeds_pandas(df)-> pd.DataFrame:
    start = datetime.datetime.now()
    print(start)
    
    #df = merge_all_speeds(analysis_date)
    
    # Categorize
    df1 = categorize_by_percentile_pandas(df, "meters_elapsed", "meters_")
    df2 = categorize_by_percentile_pandas(df1, "sec_elapsed", "sec_")
  
    # Find size of categories
    print(df2.groupby(['sec_cat','meters_cat']).size())

    # Filter out for only meters that are low or seconds that are high
    df2 = df2[(df2.meters_cat == 'meters is low') | (df2.sec_cat == 'sec is high')].reset_index(drop = True)
    print(f"{len(df2)} rows left after filtering for rows with either high seconds OR low meters") 
    
    def flag_round(row):
        if (row["meters_elapsed"] == 0) & (row["sec_elapsed"] == 0):
            return "division by 0"
        elif row["meters_cat"] == "meters is low":
            return "meters too low"
        elif row["sec_cat"] == "sec is high":
            return "seconds too high"
        else:
            return "ok"
        
    df2["flag"] = df2.apply(lambda x: flag_round(x), axis=1)
    print(df2.flag.value_counts())
    
    # Filter out for only division by 0 
    df3 = df2[(df2.flag == 'division by 0')].reset_index(drop = True)
    
    end = datetime.datetime.now()
    print(f"Took {end-start}")
    return df3

In [14]:
subset = m1[m1.shape_array_key.isin(sample_0_keys)].reset_index()

In [15]:
m2 = categorize_meters_speeds_pandas(subset)

2023-06-23 10:25:44.437503
Done with meters_
Done with sec_
sec_cat      meters_cat    
sec is avg   meters is avg     1829
             meters is high     110
             meters is low       22
sec is high  meters is avg       63
             meters is high      40
             meters is low       47
sec is low   meters is low      850
dtype: int64
1022 rows left after filtering for rows with either high seconds OR low meters
division by 0       850
seconds too high    103
meters too low       69
Name: flag, dtype: int64
Took 0:00:00.209815


In [16]:
len(m1)-len(m2)

3075512

In [17]:
len(m2)

850

In [18]:
m2.trip_id.nunique(), m1.trip_id.nunique()

(73, 72067)

In [19]:
m2.shape_array_key.nunique(), m1.shape_array_key.nunique()

(4, 4837)

In [20]:
m2._gtfs_dataset_name.nunique(), m1._gtfs_dataset_name.nunique()

(3, 76)

In [21]:
m2.groupby(["loop_or_inlining"]).agg({"shape_array_key": "nunique"})

,shape_array_key
loop_or_inlining,
0,4


In [22]:
# Number of trips that have at least one row that was divided by 0 
# for this shape array key
df1 = m2.groupby(['shape_array_key']).agg({'trip_id':'nunique'}).rename(columns = {'trip_id':'trips_with_zero'}).reset_index()

In [23]:
# Original number of trips
df2 = subset.groupby(['shape_array_key']).agg({'trip_id':'nunique'}).rename(columns = {'trip_id':'all_trips'}).reset_index()

In [24]:
df3 = pd.merge(df1, df2, how = "inner", on = 'shape_array_key')

In [25]:
df3['percent_of_trips_with_problematic_rows'] = df3.trips_with_zero/df3.all_trips * 100

In [28]:
df3

,shape_array_key,trips_with_zero,all_trips,percent_of_trips_with_problematic_rows
0,000624bd8453dbe4f2eb2765b04bcb98,14,14,100.00
1,07c9a47264a43d8d0d16ef7109e8fd68,20,20,100.00
2,0fb4f3627996269dc7075276d3b69e36,9,9,100.00
3,106d979b9a9e6338827a8e1c145e69fd,30,30,100.00


### Investigate 
#### Stage3: "vp_pared_stops"
* Keeps only first and last point of a segment.

In [29]:
def load_vp_stage3(flagged_df:pd.DataFrame, date:str) -> pd.DataFrame:
    
    # Subset the dataframe and use it to filter out for only the values of interest
    flagged_df = flagged_df[['gtfs_dataset_key', 'trip_id','stop_sequence','shape_array_key']]
    
    vp = pd.read_parquet(f"{speed_utils.GCS_PATH}vp_pared_stops_{date}")
    
    # Merge to filter
    vp2 = pd.merge(flagged_df, vp, how = "inner", on = ['gtfs_dataset_key', 'trip_id','stop_sequence','shape_array_key'])
    
    return vp2

In [30]:
vp2 = load_vp_stage3(subset, analysis_date)

In [31]:
# Check out stop sequences for the trip below that have division by 0
# subset[subset.trip_id == "1088383"].stop_sequence.unique()

In [32]:
# Stop sequences that were flagged as division by 0
# vp2[vp2.trip_id == "1088383"].sort_values(['trip_id', 'stop_sequence','location_timestamp_local'])

In [33]:
# All the stop sequences for this trip, even those that are ok
# vp_pared[vp_pared.trip_id == "1088383"].sort_values(['trip_id', 'stop_sequence','location_timestamp_local'])

In [34]:
# All the stop sequences for this trip, even those that are ok
# vp_pared[vp_pared.trip_id == "1088383"].sort_values(['location_timestamp_local','stop_sequence',])

In [35]:
def stage3_repeated_timestamps(stage3_df:pd.DataFrame)-> pd.DataFrame:
    """
    Look at how many times a time stamp is repeated a route-trip-location.
    Each of these 3 combos should have a different time for each 
    stop sequence or else the vehicle is not changing locations.
    """
    agg = (stage3_df
     .groupby(['shape_array_key','trip_id', 'location_timestamp_local'])
     .agg({'stop_sequence':'nunique'})
     .reset_index()
     .rename(columns = {'stop_sequence':'number_of_repeated_timestamps'})
    )
    
    # Only keep timestamps that are repeated more than once
    agg = (agg[agg.number_of_repeated_timestamps > 1]).reset_index(drop = True)

    return agg

In [36]:
def stage3_repeated_locations(stage3_df:pd.DataFrame):
    """
    Look at how many times a time stamp is repeated for a stop-trip-route combo.
    Each of these 3 combos should have a different location for each 
    stop sequence or else the vehicle is not changing locations.
    """
    # Concat x and y into a string
    stage3_df['pair'] = stage3_df.x.astype(str) + '/' + vp2.y.astype(str)
    
    # Count number of different stops that reference the same location
    agg = (stage3_df
     .groupby(['shape_array_key','trip_id','pair'])
     .agg({'stop_sequence':'nunique'})
     .reset_index()
     .sort_values('stop_sequence', ascending = False)
     .rename(columns = {'stop_sequence':'number_of_repeated_locs'})               
    )

    # Only keep locations that are repeated more than once
    agg = agg[agg.number_of_repeated_locs != 1].reset_index(drop = True)
    
    return agg

In [37]:
def flag_stage3(flagged_df:pd.DataFrame, date:str) -> pd.DataFrame:
    """
    Flag the errors in stage3
    """
    start = datetime.datetime.now()
    print(start)
    
    # Relevant rows from Vehicle Positions
    vp = load_vp_stage3(flagged_df, date)
    
    # Find repeated timestamps.
    multi_timestamps = stage3_repeated_timestamps(vp)
    
    # Find repeated locations
    multi_locs = stage3_repeated_locations(vp)
    
    # Merge
    timestamps_merge_cols = ['shape_array_key','trip_id','location_timestamp_local']
    loc_merge_cols =  ['shape_array_key','trip_id','pair']
    
    # Want everything found in vehicle positions, so do left merges
    m1 = (vp
          .merge(multi_timestamps, how="left", on= timestamps_merge_cols)
          .merge(multi_locs, how="left", on=loc_merge_cols)
         )
    
    drop_cols = ['vp_idx','x','y','hour','activity_date',]
    m1 = m1.drop(columns = drop_cols)
    
    # Flag
    def flag(row):
        if (row["number_of_repeated_timestamps"] > 1) & (row["number_of_repeated_locs"] > 1):
            return "repeated timestamps & locations"
        elif (row["number_of_repeated_timestamps"] > 1):
            return "repeated timestamps"
        elif (row["number_of_repeated_locs"] > 1):
            return "repeated locations"
        else:
            return "check in stage 2"
        
    m1["stage3_flag"] = m1.apply(lambda x: flag(x), axis=1)
    
    print(m1.stage3_flag.value_counts())
    
    check_in_stage2 = m1[m1.stage3_flag == "check in stage 2"]
    print(f"Have to check {len(check_in_stage2)/len(m1) * 100} % of rows in stage 2")
    
    end = datetime.datetime.now()
    print(f"Took {end-start}")
    return m1

In [38]:
m3 = flag_stage3(m2, analysis_date)

2023-06-23 10:28:50.163006
check in stage 2                   1503
repeated timestamps                 149
repeated timestamps & locations      27
repeated locations                   21
Name: stage3_flag, dtype: int64
Have to check 88.41176470588236 % of rows in stage 2
Took 0:00:15.103777


In [39]:
m3.shape

(1700, 11)

In [40]:
sort_cols = ['trip_id', 'shape_array_key', 'stop_sequence']

### Stage2: "vp_stop_segment"
* Were the right points kept?

In [41]:
subset_cols = ['gtfs_dataset_key', 'trip_id','stop_sequence','shape_array_key', 'loop_or_inlining', 'stop_id', 'meters_elapsed','sec_elapsed']

In [42]:
m_cols  = ['gtfs_dataset_key', 'stop_sequence','shape_array_key', 'loop_or_inlining','stop_id']

In [43]:
# Find rows that need to be tagged in stage 2
stage2_rows = m3[m3.stage3_flag == "check in stage 2"].reset_index()

In [44]:
stage2_routes = stage2_rows.shape_array_key.unique().tolist() 

In [45]:
# Use flagged df
stage2_rows = m2[m2.shape_array_key.isin(stage2_routes)].reset_index(drop = True)

In [46]:
# Subset df to filter the vp 
subset_for_merge = stage2_rows[subset_cols].drop_duplicates().reset_index(drop = True)

In [47]:
# What's the diff between stop segments normal/special/and without any notation?
stg2 = gpd.read_parquet(f"{speed_utils.GCS_PATH}stop_segments_{analysis_date}.parquet")

In [48]:
# Merge
stg2_m = pd.merge(stg2,
                  subset_for_merge, 
                  how = "inner",
                  on = m_cols
                 )

In [49]:
stage2_rows.shape

(850, 23)

In [50]:
# Delete out empty geo 
filtered = stg2[~stg2.geometry_arrowized.is_empty]

In [51]:
subset_for_merge.sample()

,gtfs_dataset_key,trip_id,stop_sequence,shape_array_key,loop_or_inlining,stop_id,meters_elapsed,sec_elapsed
336,db56b50ab86b5f7a4ae2fc2dd9889bbe,1089342,44,07c9a47264a43d8d0d16ef7109e8fd68,0,2308,0.00,0.00


In [52]:
# Delete out empty geometry arrowized
geo_arrowized = stg2_m[~stg2_m.geometry_arrowized.is_empty]

In [53]:
for i in ['geometry','geometry_arrowized']:
    print(f"{i}: {stg2_m[i].is_valid.sum()/len(stg2_m)}")
    print(f"{i}: {len(stg2_m[stg2_m[i].is_empty])}")

geometry: 0.9847058823529412
geometry: 0
geometry_arrowized: 1.0
geometry_arrowized: 13


#### Look at the original routes

In [54]:
# This is the original dataframe loaded in from merging stage
# It's not even flagged. 
original = pd.merge(filtered, 
                    subset[subset_cols],
                    how = "inner", 
                    on = m_cols)

In [55]:
len(geo_arrowized)

837

In [56]:
# Find number of messed up sequences...are the same sequences being hit?
(subset_for_merge
 .groupby(['shape_array_key','stop_sequence'])
 .agg({'trip_id':'nunique'})
 .rename(columns = {'trip_id':'number_of_trips_with_problematic_stop_seq'})
 .sort_values(['shape_array_key','number_of_trips_with_problematic_stop_seq']
              , ascending = False)
).head()

number_of_trips_with_problematic_stop_seq
shape_array_key                  stop_sequence                                           
106d979b9a9e6338827a8e1c145e69fd 52                                                    25
                                 46                                                    20
                                 20                                                    18
                                 48                                                    18
                                 45                                                    16

In [57]:
keep_tripid = ['1350', '1089339', '16939087', '1088383',]

#### Look at all the stop sequences vs the ones flagged as 0 for each trip

##### 106d979b9a9e6338827a8e1c145e69fd
* 1088383
* 1088403

In [58]:
# SEgments that show up have something wrong with them
geo_arrowized[geo_arrowized.trip_id == '1088383'].set_geometry("geometry_arrowized").explore('stop_sequence',  style_kwds = {'weight':10})

In [59]:
# SEgments that show up have something wrong with them
geo_arrowized[geo_arrowized.trip_id == '1088403'].set_geometry("geometry_arrowized").explore('stop_sequence',  style_kwds = {'weight':10})

In [60]:
original[original.trip_id == '1088383'].set_geometry("geometry_arrowized").explore('stop_sequence', style_kwds = {'weight':10})

#### 16939089

In [61]:
geo_arrowized[geo_arrowized.trip_id == '16939089'].set_geometry("geometry_arrowized").explore('stop_sequence', style_kwds = {'weight':10})

In [62]:
original[original.trip_id == '16939089'].set_geometry("geometry_arrowized").explore('stop_sequence', style_kwds = {'weight':10})

#### 1350

In [63]:
geo_arrowized[geo_arrowized.trip_id == '1350'].set_geometry("geometry_arrowized").explore('stop_sequence', style_kwds = {'weight':10})

In [64]:
original[original.trip_id == '1350'].set_geometry("geometry_arrowized").explore('stop_sequence', style_kwds = {'weight':10})

In [65]:
summarize(few_routes_cat, high_low_zero)

NameError: name 'summarize' is not defined

#### Draft
* Show which stops are excluded from flags
* Show how many stops are dropped
* Show % of stops that were flagged compared to total stops.

In [ ]:
def read_back_gcs():
    # Read back all the partitioned stuff - grab the file number
    # part0.parquet, part1.parquet
    start = datetime.datetime.now()
    print(f"Begin: {start}")
    gcs_file_path1 = f"{speed_utils.GCS_PATH}partitioned_flags"
    file_names_dask = extract_number(gcs_file_path1, "part")

    # https://www.geeksforgeeks.org/read-multiple-csv-files-into-separate-dataframes-in-python/
    # create empty list
    all_df = []

    # append datasets into the list
    for i in range(len(file_names_dask)):
        gcs_file_path2 = f"{gcs_file_path1}/part."
        temp_df = dd.read_parquet(f"{gcs_file_path2}{file_names_dask[i]}.parquet")
        all_df.append(temp_df)

    final_df = dd.concat(all_df, axis=0).reset_index(drop=True)
    print("Begin computing")
    final_df = final_df.compute()
    print("Done computing")
    end = datetime.datetime.now()
    print(f"Finish: {end-start}")
    return final_df

In [ ]:
def categorize_by_percentile_pandas(
    df: pd.DataFrame, column_percentile: str, column_str: str
) -> pd.DataFrame:

    # Find percentiles
    agg1 = (
        df.groupby(["shape_array_key", "stop_sequence"])[column_percentile]
        .describe(percentiles=[0.05, 0.95])
        .reset_index()
        .add_prefix(column_str)
    )
    
    # Merge 
    m1 = dd.merge(
        df,
        agg1,
        how="inner",
        left_on=["shape_array_key", "stop_sequence"],
        right_on=[
            f"{column_str}shape_array_key",
            f"{column_str}stop_sequence",
        ],
    )
    
    def percentile(row):

        if row[column_percentile] == row[f"{column_str}mean"]:
            return f"{column_str} elapsed avg"
        elif row[f"{column_str}5%"] < row[column_percentile] <= row[f"{column_str}95%"]:
            return f"{column_str} elapsed avg"
        elif row[column_percentile] <= row[f"{column_str}5%"]:
            return f"{column_str} elapsed low"
        elif row[column_percentile] > row[f"{column_str}95%"]:
            return f"{column_str} elapsed high"

        else:
            return f"{column_str} elapsed avg"
    
    
    # Apply flags
    m1[f"{column_str}cat"] = m1.apply(lambda x: percentile(x), axis=1)
    
    # Delete out any average columns
    m1 = m1.loc[m1[f"{column_str}cat"] != f"{column_str} elapsed avg"].reset_index(drop = True)
    
    # Clean
    m1[f"{column_str}cat"] = m1[f"{column_str}cat"].str.replace("_", "")
    
    columns_to_keep = [
        "shape_array_key",
        "gtfs_dataset_key",
        "_gtfs_dataset_name",
        "speed_mph",
        "loop_or_inlining",
        "stop_sequence",
        "stop_id",
        "trip_id",
        "n_trips",
        "p20_mph",
        "p80_mph",
        "p50_mph",
        "time_of_day",
        "meters_elapsed",
        "sec_elapsed",
        f"{column_str}5%",
        f"{column_str}95%",
        f"{column_str}cat",
    ]
    m1 = m1[columns_to_keep]
    print(f"Done with {column_str}")
    
    return m1  

In [ ]:
def extract_number(folder: str, phrase_to_find: str) -> list:
    """
    Extract the numeric portion of a file path.
    """
    files = find_files(folder, phrase_to_find)
    all_file_numbers = []
    for file in files:
        # https://stackoverflow.com/questions/11339210/how-to-get-integer-values-from-a-string-in-python
        file_number = "".join(i for i in file if i.isdigit())
        all_file_numbers.append(file_number)
    return all_file_numbers

In [ ]:
# Find all the parquets again
def find_files(folder: str, phrase_to_find: str) -> list:
    """
    Grab a list of files that contain the
    phrase inputted.
    """
    # Create a list of all the files in my folder
    all_files_in_folder = fs.ls(folder)
    my_files = [i for i in all_files_in_folder if phrase_to_find in i]

    # String to add to read the files
    my_string = "gs://"
    my_files = [my_string + i for i in my_files]

    # Extract digit of parquet
    return my_files

In [ ]:
def categorize_by_percentile(
    df: pd.DataFrame, column_percentile: str, column_str: str
) -> dd.DataFrame:
    
    # Find percentiles
    agg1 = (
        df.groupby(["shape_array_key", "stop_sequence"])[column_percentile]
        .describe(percentiles=[0.05, 0.95])
        .reset_index()
        .add_prefix(column_str)
    )
    
    
    # Convert to dask because it takes a very long time
    agg1_dask = dd.from_pandas(agg1, npartitions=1)
    df_dask = dd.from_pandas(df, npartitions=1)

    # Merge using dask
    merge1_dask = dd.merge(
        df_dask,
        agg1_dask,
        how="inner",
        left_on=["shape_array_key", "stop_sequence"],
        right_on=[
            f"{column_str}shape_array_key",
            f"{column_str}stop_sequence",
        ],
    )

    def percentile(row):

        if row[column_percentile] == row[f"{column_str}mean"]:
            return f"{column_str} elapsed avg"
        elif row[column_percentile] <= row[f"{column_str}5%"]:
            return f"{column_str} elapsed low"
        elif row[column_percentile] == 0:
            return f"{column_str} elapsed is 0"
        elif row[f"{column_str}5%"] < row[column_percentile] <= row[f"{column_str}95%"]:
            return f"{column_str} elapsed avg"
        elif row[column_percentile] > row[f"{column_str}95%"]:
            return f"{column_str} elapsed high"

        else:
            return "other"

    merge1_dask[f"{column_str}cat"] = merge1_dask.apply(
        lambda x: percentile(x), axis=1, meta=(f"{column_str}cat", "string")
    )
    
    # Filter for only unsually high and low stuff
    merge1_dask = merge1_dask[merge1_dask[f"{column_str}cat"].isin([f"{column_str} elapsed high", f"{column_str} elapsed low"]).reset_index(drop = True)
                              
    # Clean
    merge1_dask[f"{column_str}cat"] = merge1_dask[f"{column_str}cat"].str.replace(
        "_", ""
    )

    columns_to_keep = [
        "shape_array_key",
        "gtfs_dataset_key",
        "_gtfs_dataset_name",
        "speed_mph",
        "loop_or_inlining",
        "stop_sequence",
        "stop_id",
        "trip_id",
        "n_trips",
        "p20_mph",
        "p80_mph",
        "p50_mph",
        "time_of_day",
        "meters_elapsed",
        "sec_elapsed",
        f"{column_str}5%",
        f"{column_str}95%",
        f"{column_str}cat",
    ]
    merge1_dask = merge1_dask[columns_to_keep]
    print(f"Done with {column_str}")
    return merge1_dask

In [ ]:
def flag_round1(row):
    if (row["meters_elapsed"] == 0) & (row["sec_elapsed"] == 0):
        return "division by 0"
    elif row["meters_cat"] == "meters elapsed low":
        return "meters too low"
    elif row["seconds_cat"] == "seconds elapsed high":
        return "seconds too high"
    else:
        return "ok"
    
#def flag_round2(row):
#    if (row["meters_elapsed"] == 0) & (row["sec_elapsed"] == 0):
#        return "division by 0"
#    else:
#        return "meters/seconds are filled but flagged"

def categorize_meters_speeds_dask(df):
    start = datetime.datetime.now()
    print(f"Begin: {start}")

    # Find percentiles
    df.speed_mph = df.speed_mph.fillna(0)

    # These are now dask dataframes
    ddf_meters = categorize_by_percentile(df, "meters_elapsed", "meters_")
    ddf_seconds = categorize_by_percentile(df, "sec_elapsed", "seconds_")

    merge_cols = [
        "shape_array_key",
        "gtfs_dataset_key",
        "_gtfs_dataset_name",
        "speed_mph",
        "loop_or_inlining",
        "stop_sequence",
        "stop_id",
        "n_trips",
        "p20_mph",
        "p80_mph",
        "p50_mph",
        "meters_elapsed",
        "sec_elapsed",
        "trip_id",
        "time_of_day",
    ]

    # Merge using dask
    m1 = dd.merge(ddf_meters, ddf_seconds, how="inner", on=merge_cols)

    # Apply flags
    m1["flag"] = m1.apply(lambda x: flag_round1(x), axis=1, meta=("flag", "string"))
    print("Apply first round of flags")

    # Filter out for projects that are ok, retag for zeroes
    m2 = m1[m1.flag != "ok"].reset_index()

    # Apply flag for zeroes
    m2["flag_division_0"] = m2.apply(
        lambda x: flag_round2(x), axis=1, meta=("flag", "string")
    )
    print("Apply second round of flags")

    # Replace values in the original flag
    # https://stackoverflow.com/questions/54302694/updating-the-values-of-a-column-in-a-dask-dataframe-based-on-some-condition-on-s
    condition = m2.flag_division_0 == "division by 0"
    m2["flag"] = m2["flag"].mask(condition, m2.flag_division_0)
    print("Done flagging")

    # Print value counts
    # print(f"breakout of rows after separating out for 0: \n {m2.flag.value_counts().compute()}")

    # Filter for only projects that are divided by 0
    # m2 = m2[m2.flag == "division by 0"].reset_index()
    # Delete older column
    m2 = m2.drop(columns=["flag_division_0", "level_0", "index"])
    print("Drop columns")

    # Save
    # m2 =  m2.repartition(partition_size="5MB")
    # m2.to_parquet(f"{speed_utils.GCS_PATH}partitioned_flags", overwrite = True)
    print("Saved")

    end = datetime.datetime.now()
    print(f"Finish: {end-start}")

    return m2

In [ ]:
trips = list(equal_sampling.trip_id.unique())

In [ ]:
stops = list(equal_sampling.stop_id.unique())

In [ ]:
# Plot some of the trips
sample_data = few_routes_cat[few_routes_cat.trip_id.isin(trips)].reset_index()

In [ ]:
sample_data.shape

In [ ]:
# sample_data2 = sample_data[['shape_array_key','gtfs_dataset_key','trip_id']]

In [ ]:
plotting = sample_data.melt(
    id_vars=[
        "_gtfs_dataset_name",
        "shape_array_key",
        "trip_id",
        "stop_sequence",
        "gtfs_dataset_key",
        "loop_or_inlining",
        "n_trips",
        "meters_elapsed",
        "meters_cat",
        "seconds_cat",
        "sec_elapsed",
        "flag",
        "p20_speed_mph",
        "p80_speed_mph",
        "median_speed_mph",
    ],
    value_vars=["speed_mph"],
)

In [ ]:
# Clean
plotting = threshold_utils.pre_clean(plotting)

In [ ]:
plotting["Dropdown Menu"] = plotting["Gtfs Dataset Name"] + " " + plotting["Trip Id"]

In [ ]:
def alt_dropdown(df, col_for_dropdown: str, dropdown_menu_title: str):
    # Create dropdown menu
    # Exclude "none" operators which are only scheduled data
    df = df.loc[df[col_for_dropdown] != "None"][[col_for_dropdown]]
    dropdown_list = df[col_for_dropdown].unique().tolist()

    # Show only first operator by default
    initialize_first_op = sorted(dropdown_list)[0]
    input_dropdown = alt.binding_select(
        options=sorted(dropdown_list), name=dropdown_menu_title
    )

    selection = alt.selection_single(
        name=dropdown_menu_title,
        fields=[col_for_dropdown],
        bind=input_dropdown,
        init={col_for_dropdown: initialize_first_op},
    )

    return selection

In [ ]:
selection_test = alt_dropdown(plotting, "Dropdown Menu", "Route")

In [ ]:
(
    threshold_utils.chart_size(
        alt.Chart(plotting)
        .mark_tick(
            size=15,
            thickness=5,
        )
        .encode(
            x="Stop Sequence:N",
            y="Value:Q",
            color=alt.Color(
                "Flag:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BOLD_COLORS)
            ),
            tooltip=plotting.columns.tolist(),
        )
        .interactive(),
        1100,
        400,
    )
    .add_selection(selection_test)
    .transform_filter(selection_test)
)

In [ ]:
stop

In [ ]:
type(vehicle_positions)

In [ ]:
vehicle_positions2 = vehicle_positions[
    vehicle_positions.trip_id.isin(trips)
].reset_index()

In [ ]:
vehicle_positions2.shape

In [ ]:
gdf1 = pd.merge(
    vehicle_positions2,
    sample_data,
    how="inner",
    on=["gtfs_dataset_key", "_gtfs_dataset_name", "trip_id"],
)

In [ ]:
gdf1.shape

In [ ]:
gdf1 = gdf1[gdf1.stop_id.isin(stops)]

In [ ]:
gdf1.shape

In [ ]:
gdf1[
    [
        "geometry",
        "stop_id",
        "stop_sequence",
        "_gtfs_dataset_name",
        "shape_array_key",
        "speed_mph",
        "flag",
    ]
].explore("flag")

In [ ]:
stop

In [ ]:
high_low_zero2 = high_low_zero.melt(
    id_vars=[
        "_gtfs_dataset_name",
        "shape_array_key",
        "trip_id",
        "stop_sequence",
        "gtfs_dataset_key",
        "loop_or_inlining",
        "n_trips",
        "meters_cat",
        "seconds_cat",
        "unusual_flag",
        "time_of_day",
    ],
    value_vars=["median_speed_mph", "speed_mph", "p20_speed_mph", "p80_speed_mph"],
)

In [ ]:
high_low_zero2 = high_low_zero2.drop_duplicates(
    subset=[
        "loop_or_inlining",
        "shape_array_key",
        "stop_sequence",
        "time_of_day",
        "variable",
        "value",
    ]
).reset_index(drop=True)

In [ ]:
high_low_zero2.shape

In [ ]:
merge1.shape_array_key.nunique(), high_low_zero.shape_array_key.nunique()

In [ ]:
# Clean
high_low_zero2 = threshold_utils.pre_clean(high_low_zero2)

In [ ]:
# Add dropdown menu
high_low_zero2["Dropdown Menu"] = (
    high_low_zero2["Gtfs Dataset Name"] + " " + high_low_zero2["Shape Array Key"]
)

In [ ]:
high_low_zero2["Route Type"] = "Route Type: " + high_low_zero2[
    "Loop Or Inlining"
].astype(str)

In [ ]:
selection_test = alt_dropdown(high_low_zero2, "Dropdown Menu", "Route")

In [ ]:
# https://github.com/altair-viz/altair/issues/1168
title = (
    alt.Chart(high_low_zero2)
    .mark_text(dy=-40, size=15, fontWeight="normal")
    .encode(
        text="Route Type:N",
    )
    .add_selection(selection_test)
    .transform_filter(selection_test)
)

In [ ]:
"""total_stops_altair = (
    alt.Chart(stop_info)
    .mark_text(dy=-40, size=15, fontWeight="normal")
    .encode(
        text="Percentage Of Unusual Stops:N",
    )
    .add_selection(selection_test)
    .transform_filter(selection_test)
)"""

In [ ]:
main_chart = (
    threshold_utils.chart_size(
        alt.Chart(high_low_zero2)
        .mark_tick(
            size=15,
            thickness=5,
        )
        .encode(
            x="Stop Sequence:N",
            y="Value:Q",
            color=alt.Color(
                "Variable:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
            ),
            tooltip=high_low_zero2.columns.tolist(),
        )
        .interactive(),
        1100,
        400,
    )
    .add_selection(selection_test)
    .transform_filter(selection_test)
)

In [ ]:
high_low_zero2.shape

In [ ]:
(title & total_stops_altair | main_chart)

In [ ]:
high_low_zero.shape_array_key.unique()

In [ ]:
chart2 = threshold_utils.chart_size(chart2, 75, 200)

In [ ]:
chart2 = chart2.add_selection(selection_test).transform_filter(selection_test)

In [ ]:
title = threshold_utils.chart_size(title, 20, 20)

In [ ]:
alt.data_transformers.enable("default", max_rows=None)

In [ ]:
title & (chart1.interactive() & chart2.interactive())